In [1]:
# load TensorBoard
%load_ext tensorboard

# imports
import datetime
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

from CustomData import CustomData
from NeuralNet import NeuralNet

In [2]:
# parameters
EPOCHS=10
BATCH_SIZE=128

In [ ]:
# generate custom data
data_gen = CustomData()

x_train, y_train = data_gen.generate_data(n=10000)
x_test, y_test = data_gen.generate_data(n=2000)


In [3]:
# preprocessing
x_train, y_train, labels_train = data_gen.preprocessing(x_train, y_train)
x_test, y_test, labels_test = data_gen.preprocessing(x_test, y_test)

# input shape and number of classes
inp_shape = x_train.shape
num_classes = len(labels_train)

In [4]:
# initialize model
model = NeuralNet(num_classes)

In [5]:
# Callbacks
# early stopping
early_stopping = EarlyStopping(monitor="loss", patience=2)
# tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = TensorBoard(log_dir=log_dir, histogram_freq=1)

callbacks =[early_stopping, tensorboard_cb]

In [6]:
# train model
history = model.comp_and_fit(x_train, y_train, EPOCHS, BATCH_SIZE, callbacks)

Epoch 1/5
63/63 - 32s - loss: 0.2087 - accuracy: 0.9119 - val_loss: 0.0160 - val_accuracy: 0.9935
Epoch 2/5
63/63 - 33s - loss: 0.0156 - accuracy: 0.9945 - val_loss: 0.0036 - val_accuracy: 0.9985
Epoch 3/5
63/63 - 31s - loss: 0.0080 - accuracy: 0.9975 - val_loss: 0.0033 - val_accuracy: 0.9985
Epoch 4/5
63/63 - 31s - loss: 0.0088 - accuracy: 0.9967 - val_loss: 0.0642 - val_accuracy: 0.9780
Epoch 5/5
63/63 - 31s - loss: 0.0118 - accuracy: 0.9969 - val_loss: 8.1789e-04 - val_accuracy: 1.0000


In [10]:
%tensorboard --logdir logs

Launching TensorBoard...

In [8]:
# model evaluation
loss, accuracy = model.test_eval(x_test, y_test)
print("Accuracy:", round(accuracy*100, 2), "%")


Accuracy: 99.8 %
